In [65]:
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import tokenizer
import numpy as np
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import re
import numpy as np
import multiprocessing 
from multiprocessing import Pool
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
# Import required modules
import pandas as pd
from sklearn import preprocessing


[nltk_data] Downloading package punkt to /home/dval/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
#Upload file with google-translated hedonometer words and scores from hedonometer.org (translated by google translate)
file = pd.read_csv("hedonometer_urdu_pashto.csv")

In [68]:
#Create a dictionary of pashto scores
scores = file[["Pashto", "Score"]]
scores.columns = ["word", "score"]
scores = dict(zip(scores.word, scores.score))

In [70]:
#Create a dictionary of pashto standard deviations
sds = file[["Pashto", "Standard Deviation"]]
sds.columns = ["word", "deviation"]
sds = dict(zip(sds.word, sds.deviation))

In [71]:
#open kp pashto tweets
with open('kp_tweets_pashto.pickle', "rb") as f:
    tweets = pickle.load(f)

In [72]:
#Define formula to determine sentiment
def sentiment(sentence):
    senti=0
    total=0
    words = sentence.split(" ")
    for word in words:
        if word in scores.keys():
            senti += scores[word]
            total += 1
    try:
        return senti/total
    except ZeroDivisionError:
        return 0

In [36]:
#Define forumula to apply to dataframe
def return_sentiment(data):
    return pd.Series(data["text"].apply(lambda x: sentiment(x)))

In [37]:
#Apply formula to parallelize across cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [38]:
#Calculate pashto sentiment
tweets["hedonometer"] = parallelize_dataframe(tweets, return_sentiment)

In [39]:
#Define formula to calculate sd of scores
def standard_deviation(sentence):
    standard_deviation=0
    total=0
    words = sentence.split(" ")
    for word in words:
        if word in sds.keys():
            standard_deviation += sds[word]
            total += 1
    try:
        return standard_deviation/total
    except ZeroDivisionError:
        return 0

In [40]:
#Define formula to apply to dataframe
def return_standard_deviation(data):
    return pd.Series(data["text"].apply(lambda x: standard_deviation(x)))

In [41]:
#Define formula to parallelize across cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [42]:
#Calculate sd of pashto tweets
tweets["hedo_sd"] = parallelize_dataframe(tweets, return_standard_deviation)

In [43]:
#Define a formula to select the words used to define sentiment
def grab_hits(sentence):
    hits=[]
    words = sentence.split(" ")
    for word in words:
        if word in sds.keys():
            hits.append(word)
    return hits

In [44]:
#Define formula to apply it to dataframe
def return_hits(data):
    return pd.Series(data["text"].apply(lambda x: grab_hits(x)))

In [45]:
#Define formula to parallelize across cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [46]:
#Grab words used to determine sentiment and save in column
tweets["hits"] = parallelize_dataframe(tweets, return_hits)

In [48]:
#Save file
with open("NEW_non_kp_hedonometer_pashto.pickle", "wb") as f:
    pickle.dump(tweets, f)

In [ ]:
#REPEAT FOR KP URDU, NON-KP PASHTO AND NON-KP URDU